# Introduction to Dynamic Circuits and QEC with Qiskit

**Qiskit Slack: @Quantom & @Micheal Healy & Ed Chen**

**Github: taalexander & @mbhealy & @ehchen**

Before running this tutorial make sure you are setup with the [Hello IEEE notebook](./hello-ieee.ipynb).

- This tutorial introduces pre-release dynamic circuit support through Qiskit on IBM Quantum hardware. This will change quickly as we develop this functionality, with general release coming by IBM Quantum Summit.
- We will run some very simple error-correction in the form of a bit-flip code.
- At the end we will complete some exercises, the solutions may be found in the [solution notebook](./ieee-qec-tutorial-2022-solutions.ipynb).

<img src="figs/ibm_quantum.png" alt="drawing" style="width:1000px;"/>

In [ ]:
import os
from typing import Any, Dict, List, Optional, Union

import numpy as np
import matplotlib.pyplot as plt

from qiskit import IBMQ, QuantumCircuit, QuantumRegister, ClassicalRegister, quantum_info as qi
from qiskit.providers.ibmq import RunnerResult
from qiskit.result import marginal_counts
import qiskit.tools.jupyter

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")


In [ ]:
%load_ext autoreload
%autoreload 2

## Running dynamic circuits with Qiskit

The first step we need to take is to instantiate our Qiskit runtime service 
instance and load our backend. This gives us a handle to the hardware we will
control.

For today's tutorial we have reserved access to a high-quality backend.

Make sure to use hub/group/provider below.

In [ ]:
# Note: This can be any hub/group/project that has access to the required device and the Qiskit runtime.
# Verify that ``qasm3`` is present in ``backend.configuration().supported_features``.
hub = "ibm-q-community"
group = "ieee-demos"
project = "main"
backend_name = "ibm_peekskill"
hgp = f"{hub}/{group}/{project}"

In [ ]:
from qiskit.circuit import Delay, Parameter
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(instance=hgp)
backend = service.backend(backend_name, instance=hgp)

In [ ]:
# Addresses a bug in the current Qiskit runtime package
# that will be fixed on the next Qiskit release
target = backend.target
if "delay" not in target:
    target.add_instruction(
        Delay(Parameter("t")), {(bit,): None for bit in range(target.num_qubits)}
    )

As there may be many tutorial attendees contending for access to the hardware
we may also run the tutorial on the simulator below.

Make sure to set the variable `sim = True` below to enable this functionality
throughout the notebook.

In [ ]:
from qiskit.providers.aer import AerSimulator, Aer
from qiskit.providers.aer.noise import NoiseModel

backend_noise_model = NoiseModel.from_backend(backend)
#backend_sim = AerSimulator(noise_model=backend_noise_model)
backend_sim = AerSimulator()
ideal_sim = Aer.get_backend('qasm_simulator')

## Visualize the backend
Below we see a visualization of the hardware backend. For a more detailed overview see
[ibm_peekskill on IQS](https://quantum-computing.ibm.com/services/resources?search=peeks&system=ibm_peekskill).

Take note that the average T2* time is over 300us and average CNOT error is 0.0056.

In [ ]:
import utils
backend

### Set some global execution properties for the notebook.

Feel free to adjust these.

In [ ]:
shots: int = 1024 # Number of shots to run each circuit for

init_num_resets: int = 3 # Set the number of resets to initialize qubits between circuits
init_delay: float = 0. # Set the initialization idle time for qubits between circuits

sim: bool = False # Set True to simulate our experiments and False to run in hardware
dynamical_decoupling: bool = False # Set True to enable dynamical decoupling
    
favourite_qubits: List[int] = [5, 3, 8, 2, 9]

In [ ]:
# Some general helper routines that will be used throughout the notebook

import mapomatic as mm
from qiskit.circuit.library import XGate
from qiskit.transpiler import PassManager
from qiskit.transpiler.instruction_durations import InstructionDurations
from qiskit_ibm_runtime import RuntimeJob, IBMBackend

def convert_cycles(time_in_seconds: float, backend: IBMBackend) -> int:
    cycles = time_in_seconds / (backend.configuration().dt)
    return int(cycles + (16 - (cycles % 16)))

def execute(circuits: List[QuantumCircuit], verbose: bool = True, **kwargs) -> RuntimeJob:
    """A helper method to execute circuits with common settings in the notebook.
    
    Args:
        circuits: To execute.
        verbose: Emit execution information.
        sim: Run with simulator instead of hardware. By default reads global variable "sim"
    
    """
    if isinstance(circuits, QuantumCircuit):
        circuits = [circuits]

    if sim:
        return backend_sim.run(circuits, shots=shots, **kwargs)
    else:
    
        job = run_openqasm3(circuits, backend, verbose=False, shots=shots, init_num_resets=init_num_resets, init_delay=init_delay, **kwargs)
        if verbose:
            print(f"Running on qasm3, job id: {job.job_id}")

        return job
    
def calculate_initial_layout(
    circuit: QuantumCircuit, 
    mapomatic: Optional[bool] = None, 
    favourite_qubits: Optional[List[int]] = None, 
    blacklist_qubits: Optional[List[int]] = None
) -> List[int]:
    """Routine to help choose the ideal qubit layout given an input circuit.
    
    Args:
        circuit: To evaluate layout for.
        mapomatic: Use mapomatic to choose ideal qubits, otherwise naievly choose ``favourite_qubits``
        favourite_qubits: A list of favourite qubits to ensure are in the layout
        blacklist_qubits: Qubits that are not allowed in the layout.
    """
    deflated = mm.deflate_circuit(circuit)
    if mapomatic:
        deflated = mm.deflate_circuit(circuit)
        available_layouts = [layout for layout in mm.matching_layouts(deflated, backend.configuration().coupling_map, strict_direction=False) if layout[:len(favourite_qubits)] == favourite_qubits and not set(layout).intersection(set(blacklist_qubits))]
        scores = mm.evaluate_layouts(deflated, available_layouts, backend)
        return scores[0][0]
    
    if favourite_qubits is None:
        favourite_qubits = []
    
    if blacklist_qubits is None:
        blacklist_qubits = []
        
    return (favourite_qubits + list(set(range(backend.num_qubits)) - set(favourite_qubits)))[:deflated.num_qubits]

def apply_dynamical_decoupling(
    circuits: List[QuantumCircuit], 
    backend: IBMBackend, 
    initial_layout: List[int],
) -> List[QuantumCircuit]:
    """Apply dynamic circuit dynamical decoupling."""

    from qiskit_ibm_provider.transpiler.passes.scheduling import DynamicCircuitScheduleAnalysis, PadDynamicalDecoupling
    dd_sequence = [XGate(), XGate()]
    durations = InstructionDurations.from_backend(backend)
    pm = PassManager(
        [
            DynamicCircuitScheduleAnalysis(durations),
            PadDynamicalDecoupling(durations, dd_sequence, qubits=initial_layout),
        ]
    )
    return pm.run(circuits)

def apply_transpile(
    circuits: List[QuantumCircuit], 
    backend: IBMBackend, 
    initial_layout: List[int], 
    dynamical_decoupling: bool = dynamical_decoupling, 
    **transpile_kwargs,
) -> List[QuantumCircuit]:
    """Fixed transpile routine to be used for consistency throughout the notebook."""

    transpiled_circuits = transpile(circuits, backend, initial_layout=initial_layout, optimization_level=3, **transpile_kwargs)
    if dynamical_decoupling:
        return apply_dynamical_decoupling(transpiled_circuits, backend, initial_layout)
    return transpiled_circuits

# Dynamic circuit support on IBM hardware

We want to provide our users the tools to explore techniques such as quantum error correction, early stopping / repeat until success and circuit knitting. We have designed a next-generation control-systems architecture that will enable
us to scale to thousands of qubits while providing real-time capabilities in hardware.

Below we will show an early glimpse of how to access these capabilities.

*Note*: run_openqasm3 is only a temporary function. It will be replaced by `backend.run` by the IBM Quantum Summit

<img src="figs/control_system.png" alt="drawing" style="width:1000px;"/>

### Writing a simple circuit with control-flow

Below we write a circuit that resets the state of our qubit based on a measurement result.

In [ ]:
qubit = favourite_qubits[0]
print(f"We will use qubit {qubit}")

In [ ]:
# Exercise

from utils import run_openqasm3

from qiskit import QuantumRegister, ClassicalRegister, transpile

qr = QuantumRegister(1)
crx = ClassicalRegister(1, name="xresult")
crm = ClassicalRegister(1, name="measureresult")
qc_reset = QuantumCircuit(qr, crx, crm, name="Reset")

# Complete the circuit to measure and conditionally reset the target
# qubit.

qc_reset = transpile(qc_reset, backend)
qc_reset.draw(output="mpl", idle_wires=False)

In [ ]:
# Solution

from utils import run_openqasm3

from qiskit import QuantumRegister, ClassicalRegister, transpile

qr = QuantumRegister(1)
crx = ClassicalRegister(1, name="xresult")
crm = ClassicalRegister(1, name="measureresult")
qc_reset = QuantumCircuit(qr, crx, crm, name="Reset")
qc_reset.x(0)
qc_reset.measure(0, crx)
qc_reset.x(0).c_if(crx, 1)
qc_reset.measure(0, crm)

qc_reset = transpile(qc_reset, backend)
qc_reset.draw(output="mpl", idle_wires=False)

### Now, let's run

In [ ]:
reset_job = run_openqasm3(qc_reset, backend, verbose=True, shots=shots, init_num_resets=0, init_delay=0) # Turn off automatic init

### And interpret the results

In [ ]:

from qiskit.result import marginal_counts

reset_result = reset_job.result()
reset_counts = reset_result.get_counts(0)
# Look at only the final measurement result
initial_measurement_result = marginal_counts(reset_counts, indices=[0])
mitigated_reset_results = marginal_counts(reset_counts, indices=[1])
print(f"Full counts including reset: {reset_counts}")
print(f"Conditional X gate applied on: {initial_measurement_result['1']} shots")
print(f"Results from our reset - |0\rangles prepared: {mitigated_reset_results.get('0')}, |1\rangles prepared: {mitigated_reset_results.get('1', 0)}"
)

### What is going on under the hood?

We are submitting circuit jobs to a [Qiskit Runtime](https://quantum-computing.ibm.com/lab/docs/iql/runtime/) [qasm3-runner program](https://quantum-computing.ibm.com/services/programs?search=qasm3&program=qasm3-runner) that exposes a ``backend.run(circuits, ...)`` like interface, but which accepts either ``QuantumCircuit`` or OpenQASM 3 strings for ``circuits``.

These circuits are processed in the runtime and then submitted to our next-generation systems software stack which compiles the program for our real-time control systems and then launches the output executable within the hardware runtime.

**Note**: In a future release of the `qiskit-ibm-provider`, we will release an updated interface in the form of `backend.run(circuit)` which invokes the Qiskit Runtime.

![Qiskit Runtime](https://iqx-docs.quantum-computing.ibm.com/_images/Qiskit_Runtime_architecture1.png)

### Submit an OpenQASM 3 string directly

It is possible to submit an OpenQASM3 string directly to the runtime. This is useful for the cases where the `QuantumCircuit` does not yet support all of the semantics of OpenQASM 3. To see what features of OpenQASM 3 are currently supported in hardware, please see the [feature support table here](https://quantum-computing.ibm.com/services/docs/services/manage/systems/dynamic-circuits/feature-table).

We can get the OpenQASM 3 for our circuit from Qiskit.

**Note**: We current do not merge your circuits into a single execution when OpenQASM 3 source strings are submitted directly. In this case each OpenQASM 3 source will be run as an individual execution and will take significantly longer to execute in the hardware.

### We can use Qiskit to emit a QuantumCircuit as an OpenQASM3 source string

In [ ]:
from qiskit import qasm3

basis_gates = backend.configuration().basis_gates

def dump_qasm3(circuit: QuantumCircuit, backend: IBMBackend = backend) -> str:
    return qasm3.Exporter(includes=[],
                          basis_gates=backend.configuration().basis_gates,
                          disable_constants=True).dumps(circuit)

qasm3_reset = dump_qasm3(qc_reset)
print(qasm3_reset)

### We can then submit this directly to the runtime
For the rest of the tutorial we will use the method `run_openqasm3` which is a simple wrapper around the runtime calls above with some additional logging.

In [ ]:
runtime_job = run_openqasm3(qasm3_reset, backend, verbose=True, shots=shots, init_num_resets=0, init_delay=0)

### OpenQASM 3 gives us more classical compute
The semantics of OpenQASM 3 enable more classical compute than is currently
available in Qiskit. For now we can access this by submitting an OpenQASM3 source string directly.

In [ ]:
classical_compute_qasm3 = """
OPENQASM 3.0;

bit a;
bit b;
bit c;

x $0;
x $2;

a = measure $0; // expected "1"
b = measure $1; // expected "0"
c = measure $2; // expected "1"

bit[3] d = "100";

if (((a | b) & c) == 1) {
    // Path will nearly always execute 
    d[0] = 1;
} else {
    // Path will rarely execute outside of SPAM errors
    d[0] = 0;
}

bit[3] e = "101";
// Conditionally execute based on classical bit array comparison
// Should always execute
if (d == e) {
    x $1;
}

bit final;
final = measure $1; // expected "1"
"""

classical_compute_job = run_openqasm3(classical_compute_qasm3, backend, verbose=False, shots=shots)

## Teleport a quantum state

One example of a technique that is enabled by having control-flow available in a circuit is
quantum teleportation. 

Visit the Qiskit Textbook to know more about [Quantum teleportation](https://qiskit.org/textbook/ch-algorithms/teleportation.html).

<img src="figs/telamon.jpeg" alt="drawing" style="width:800px;"/>

### Step 1: Prepare the teleportation circuit

The circuit below prepares the state

$$ |\Psi\rangle = \frac{1}{2}( |00\rangle(\alpha|0\rangle+\beta|1\rangle) + |01\rangle(\alpha|0\rangle+\beta|1\rangle) + |10\rangle(\alpha|0\rangle-\beta|1\rangle) +|11\rangle(\alpha|1\rangle-\beta|0\rangle)) $$

If we measure qubits $\$0$ and $\$1$ we project the state to one of $\{\alpha|0\rangle+\beta|1\rangle, \alpha|1\rangle+\beta|0\rangle, \alpha|0\rangle-\beta|1\rangle, \alpha|1\rangle-\beta|0\rangle\}$ with corresponding measurement bitstrings of $\{00, 01, 10, 11\}$.

Each of these states is only a $Z$ or $X$ operation away from the original input state $\alpha|0\rangle+\beta|1\rangle$.
By using dynamic circuit capabilities we can apply this "correction" sequence.

We will perform similar operations to implement the bit-flip code below.

In [ ]:
qr = QuantumRegister(3)
cr = ClassicalRegister(3, name="result")
crz = cr[0]
crx = cr[1]
result = cr[2]

qc_teleport = QuantumCircuit(qr, cr, name="Teleport")

# Apply teleportation circuit
qc_teleport.h(qr[1])
qc_teleport.cx(qr[1], qr[2])
qc_teleport.cx(qr[0], qr[1])
qc_teleport.h(qr[0])
qc_teleport.measure(qr[0], crz)
qc_teleport.measure(qr[1], crx)
qc_teleport.z(qr[2]).c_if(crz, 1)
qc_teleport.x(qr[2]).c_if(crx, 1)
qc_teleport.draw(output="mpl")

### Step 2: Prepare the state to be teleported

For our demonstration we need to choose and prepare a state to teleport. For simplicity we look at $|1\rangle$.

In [ ]:
# Prepare |1> for teleportation
qc_state_prep = QuantumCircuit(1)
qc_state_prep.x(0)
print(qc_state_prep)

target_state_prep = qi.Statevector.from_instruction(qc_state_prep)
target_state_prep.draw(output="bloch")

### Step 3: Compose the state initialization with the teleportation

In [ ]:
teleport_qubits = favourite_qubits[:3]

qc_teleport_state = QuantumCircuit(qr, cr, name="Teleport Hadamard")

# Prepare state to teleport
qc_teleport_state.compose(qc_state_prep, [qr[0]], inplace=True)
qc_teleport_state.barrier(qr)

# Compose with teleportation circuit
qc_teleport_state.compose(qc_teleport, inplace=True)

qc_teleport_state.draw(output="mpl")

### Step 4: Transpile for backend 

In [ ]:
qc_teleport_experiment = qc_teleport_state.copy()
qc_teleport_experiment.measure(qr[2], result)
qc_teleport_experiment = apply_transpile(qc_teleport_experiment, backend, initial_layout=teleport_qubits)
qc_teleport_experiment.draw(output="mpl", idle_wires=True)

### And execute

In [ ]:
teleport_job = execute(qc_teleport_experiment, verbose=False)

In [ ]:
teleport_result = teleport_job.result()
print(f"All teleportation counts: {teleport_result.get_counts(0)}")

marginal_teleport_counts = marginal_counts(teleport_result.get_counts(0), indices=[2])
print(f"Marginalized teleportation counts: {marginal_teleport_counts}")

## Running basic quantum error correction

*Note*: This tutorial draws heavily from these excellent tutorials [1](https://qiskit.org/textbook/ch-quantum-hardware/error-correction-repetition-code.html) [2](https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/awards/teach_me_quantum_2018/intro2qc/10.Quantum%20error%20correction.ipynb).

A few more excellent references for an introduction to quantum error correction [3](https://www2.physics.ox.ac.uk/sites/default/files/ErrorCorrectionSteane06.pdf) [4](https://arxiv.org/abs/1907.11157) [5](https://arxiv.org/abs/0905.2794) [6](https://www.nature.com/articles/s41534-020-00304-y)

### The basics of error correction

The basic ideas behind error correction are the same for quantum information as for classical information. This allows us to begin by considering a very straightforward example: speaking on the phone. If someone asks you a question to which the answer is 'yes' or 'no', the way you give your response will depend on two factors:

* How important is it that you are understood correctly?
* How good is your connection?

Both of these can be parameterized with probabilities. For the first, we can use $P_a$, the maximum acceptable probability of being misunderstood. If you are being asked to confirm a preference for ice cream flavours, and don't mind too much if you get vanilla rather than chocolate, $P_a$ might be quite high. If you are being asked a question on which someone's life depends, however, $P_a$ will be much lower.


For the second we can use $p$, the probability that your answer is garbled by a bad connection. For simplicity, let's imagine a case where a garbled 'yes' doesn't simply sound like nonsense, but sounds like a 'no'. And similarly a 'no' is transformed into 'yes'. Then $p$ is the probability that you are completely misunderstood.

A good connection or a relatively unimportant question will result in $p<P_a$. In this case it is fine to simply answer in the most direct way possible: you just say 'yes' or 'no'.

If, however, your connection is poor and your answer is important, we will have $p>P_a$. A single 'yes' or 'no' is not enough in this case. The probability of being misunderstood would be too high. Instead we must encode our answer in a more complex structure, allowing the receiver to decode our meaning despite the possibility of the message being disrupted. The simplest method is the one that many would do without thinking: simply repeat the answer many times. For example say 'yes, yes, yes' instead of 'yes' or 'no, no no' instead of 'no'.

If the receiver hears 'yes, yes, yes' in this case, they will of course conclude that the sender meant 'yes'. If they hear 'no, yes, yes', 'yes, no, yes' or 'yes, yes, no', they will probably conclude the same thing, since there is more positivity than negativity in the answer. To be misunderstood in this case, at least two of the replies need to be garbled. The probability for this, $P$, will be less than $p$. When encoded in this way, the message therefore becomes more likely to be understood. The code cell below shows an example of this.

In [ ]:
p1 = 0.01
p3 = 3 * p1**2 * (1-p1) + p1**3 # probability of 2 or 3 errors
print('Probability of a single reply being garbled: {}'.format(p1))
print('Probability of a majority of the three replies being garbled: {:.4f}'.format(p3))

Say, $P_a=p1$, that our acceptance probability is just at the threshold of a single reply being garbled. With the majority voting technique above $p3<P_a$, so this technique solves our problem. If it had not, then we can simply add more repetitions. The fact that $p3<P_a$ above comes from the fact that we need at least two replies to be garbled to flip the majority, and so even the most likely possibilities have a probability of $\sim p1^2$. For five repetitions we'd need at least three replies to be garbled to flip the majority, which happens with probability $\sim p1^3$. The value for $P$ in this case would then be even lower. Indeed, as we increase the number of repetitions, $P$ will decrease exponentially. No matter how bad the connection, or how certain we need to be of our message getting through correctly, we can achieve it by just repeating our answer enough times.

Though this is a simple example, it contains all the aspects of error correction.
* There is some information to be sent or stored: In this case, a 'yes' or 'no'.
* The information is encoded in a larger system to protect it against noise: In this case, by repeating the message.
* The information is finally decoded, mitigating for the effects of noise: In this case, by trusting the majority of the transmitted messages.

This same encoding scheme can also be used for binary, by simply substituting `0` and `1` for 'yes' and 'no'. It can therefore also be easily generalized to qubits by using the states $\left|0\right\rangle$ and $\left|1\right\rangle$. In each case it is known as the *repetition code*. Many other forms of encoding are also possible in both the classical and quantum cases, which outperform the repetition code in many ways. However, its status as the simplest encoding does lend it to certain applications.

## Quantum error correction

While repetition conceptually underlies the codes we will implement in this notebook implementing the repetition code naively will not allow us to store *quantum information*. This is because the very act of measuring our data qubits will destroy the encoded state. For example consider that we encoded the state $|\Psi_0\rangle = \alpha |0\rangle + \beta |1\rangle$ as $|\tilde{\Psi}_0\rangle = \alpha |000\rangle + \beta |111\rangle$ if we measure all of the qubits according to the Born rule we will obtain the outcome states with probability $Pr(|000\rangle) = |\alpha|^2$ and $Pr(|111\rangle) = |\beta|^2$. 


If we then decode our encoded state $|\tilde{\Psi}_1\rangle \rightarrow |\Psi_1\rangle$ we will obtain

$$
  |\Psi_1\rangle =
\begin{cases}
|0\rangle & \propto Pr(|000\rangle) = |\alpha|^2\\
|1\rangle & \propto Pr(|111\rangle) = |\beta|^2
\end{cases}
$$

Importantly this is **not** the state we encoded $|\Psi_0\rangle$. This is because our measurement operation *does not commute with the encoding of the state*.

At face value this might imply that quantum error correcting codes are not possible. However, it turns out we can exploit additional ancilla qubits and entanglement to measure what are known as *stabilizers* that do not transform our encoded quantum information, while still informing us of some classes of errors that may have occurred. 

## Stabilizer codes

A quantum stabilizer code encodes $k$ logical qubits into $n$ physical qubits. The coding rate is defined as the ration of $k/n$. The *stabilizer* $S$ is an Abelian subgroup of the Pauli group $\Pi^n$ (importantly it does not matter in which order we apply our stabilizers). The +1 eigenspace of the stabilizer operators make up the *codespace* of the code. It will have dimension $2^k$ and can therefore encode $k$ qubits. 

Stabilizer codes critically focus on correcting a discrete error set with support from the Pauli group $\Pi^n$. Assume the set of possible errors are $ \epsilon \subset \Pi^n$. For example in a bit flip code with three-qubits encoding the quantum state we will have $\epsilon = \{IIX, IXI, XII\}$.

As both $S$ and $\epsilon$ are subsets of the Pauli group $\Pi^n$ if an error $E \in \epsilon$ is applied to a state it will either commute or anticommute with one of the generators of the Stabilizer group $g \in S$. If the error anticommutes with a Stabilizer element it will be both detectable and correctable as it will change the sign of the stabilizer measurement.

In general we can measure each element of the generator of the stabilizer (the minimal representation) $\{g_1, \cdots, g_{n-k} | \forall i \in \{1, \cdots, n-k\}, g_i \in S\}$ which will produce a syndrome bitstring $\vec{x}$ where we assign `0` for a generator that commutes (+1 eigenspace) and `1` for a generator that anticommutes (-1 eigenspace). This is known as measuring the error *syndrome*.

If $E_a$ commutes with $g_i$:
$$g_iE_a|\tilde{\Psi}\rangle = E_ag_i|\tilde{\Psi}\rangle=E_a|\tilde{\Psi}\rangle$$

If $E_a$ anti-commutes with $g_i$:
$$g_iE_a|\tilde{\Psi}\rangle = -E_ag_i|\tilde{\Psi}\rangle=-E_a|\tilde{\Psi}\rangle$$


Measuring the stabilizer generators *will not* modify our encoded qubit as by definition they commute with the codespace, however, *errors will*.

## Correcting errors

We now know how to *detect* an error (by measuring the stabilizers and observing their eigenvalues). 

The next step is to *decode* and *correct* the error.

The condition for recovery is either: (1) $E_a^\dagger E_b \in S$ or (2) there exists $M \in S$ that anticommutes with $E_a^\dagger E_b$. [For more details, see here.](http://theory.caltech.edu/~preskill/ph229/notes/chap7.pdf)

While the above introduction is a mathematical introduction to Stabilizer codes, we believe its better to learn the practicalities of implementing them with some hands on practice.


### Stabilizer codes in practice
In general there is a common structure to most experiments with stabilizer codes. While someday we will write a *logical* program and have the hardware determine how to encode and decode syndromes to correct errors in the program. Today, we are just starting to explore the practical implementation of QEC. Therefore we manually encode our logical state in *physical qubits* with circuits that we write at the physical qubit level. When we do this there is typically a standard flow to such an experiment:

1. Initialize our input physical state $|\Psi_0\rangle \rightarrow |\Psi_1\rangle$ we wish to protect.
2. Encode our state in our Stabilizer's codespace as $|\Psi_1\rangle \rightarrow |\tilde{\Psi}_1\rangle$.
3. Apply an error-channel. These may be simulated Krauss maps, probabilitiscally applied gates (as we demonstrate below), or simply the passive error-channel in the device of interest. $|\tilde{\Psi_1}\rangle \rightarrow |\tilde{\Psi}_2\rangle$.
4. Measure the syndrome by measuring the generators of our stabilizer $\langle\tilde{\Psi}_2|g_0\cdots g_n|\tilde{\Psi}_2\rangle \rightarrow |\tilde{\Psi}_2\rangle,\ \  \vec{x}$
5. Apply a decoding sequence $D(\vec{x})$ to our stabilizer and apply the correction sequence for the error (if possible) that we have decoded, $D(\vec{x})|\tilde{\Psi}_2\rangle \rightarrow |\tilde{\Psi}_1\rangle$. It is in computing $D(\vec{x})$ where dynamic circuit capabilities are important.
6. Loop to 3, if we are running multiple iterations of the Stabilizer sequence.
7. Decode our encoded state to determine whether the encoded state was corrupted.
8. Measure the final data qubit to observe the state we protected and determine how well the code performed.

## Executing the bit-flip code on hardware

The bit-flip code is among the simplest examples of a stabilizer code. It can protect our state against a single bit-flip (X) error on any of the encoding qubits. if we consider the action of bit-flip error $X$ which maps $|0\rangle \rightarrow |1\rangle$ and $|1\rangle \rightarrow |0\rangle$ on any of our qubits we have $\epsilon = \{E_0, E_1, E_2 \} = \{IIX, IXI, XII\}$. The code requires five-qubits, three of which are used to encode the protected state and the remaining two which are used as stabilizer measurement ancillas. Nonetheless, the ancillas are not counted so this means our coding rate for this state is $1/3$.

In [ ]:
# Approximate duration of the measurement processing / conditional latency
block_branch_duration = 0.5e-6 
block_branch_cycles = convert_cycles(block_branch_duration, backend)

In [ ]:
# Setup a base quantum circuit for our experiments
qreg_data = QuantumRegister(3)
qreg_measure = QuantumRegister(2)
creg_data = ClassicalRegister(3)
creg_syndrome = ClassicalRegister(2)
state_data = qreg_data[0]
ancillas_data = qreg_data[1:]

def build_qc() -> QuantumCircuit:
    return QuantumCircuit(qreg_data, qreg_measure, creg_data, creg_syndrome)

### Initialize our qubit
To protect a quantum state we must first prepare it!
In general we can prepare the state $$|\Psi_0\rangle \rightarrow |\Psi_1\rangle = (\alpha |0\rangle + \beta |1\rangle)$$
In the circuit below we prepare the physical state $$|\Psi_1\rangle = |1\rangle$$

In [ ]:
# Exercise

qc_init = build_qc()

# Complete to initialize the |1> state
qc_init.barrier(qreg_data)

qc_init.draw(output="mpl")

In [ ]:
# Solution

qc_init = build_qc()

qc_init.x(qreg_data[0])
qc_init.barrier(qreg_data)

qc_init.draw(output="mpl")

### Encode our logical state
To protect our qubit we must encode it in the codespace. For the case of the bit-flip code this
is very similar to the repetition code, where we implement repetition using the entangling
`CX` gate rather than a classically conditioned bit-flip as we would do in the classical case.

The encoding circuit below will map $|\Psi_1\rangle = U_{en}|\Psi_0\rangle = |00\rangle (\alpha |000\rangle + \beta |111\rangle)$

The codespace of our bitflip code is therefore $V_s = \{|000\rangle, |111\rangle\}$.

The stabilizers for the bit flip code is $S = III, IZZ, ZIZ, ZZI$. 


Operationally what this means is that
a single bit-flip error applied to the qubits will modify the observed state of a stabilizer measurement but leave it within the codespace. It is also straightforward to show that any two non-trivial Stabilizer elements can generate the full stabilizer. For example taking the generator set $G_0 = \{g_0, g_1 \} = \{ IZZ, ZIZ \}$

We can see this as $g_0g_1 = IZZ . ZIZ = ZZI$ and $g_0g_0 = IZZ.IZZ = III$ completing our Stabilizer.

This means that we must only measure our two generators $g_0$ and $g_1$ to detect any *correctable* error.

It is easy to see the prepared state is a +1 eigenstate of our stabilizers
$IZZ |\Psi_1\rangle = |\Psi_1\rangle$ 
$ZIZ |\Psi_2\rangle = |\Psi_2\rangle$ 

This is because the Stabilizer measures the *parity* of the two target qubits.


The circuit below maps
$$|\Psi_1\rangle \rightarrow |\tilde{\Psi}_1\rangle$$

In [ ]:
# Exercise

# Complete the method below to encode the bit-flip code
def encode_bit_flip(qc, state, ancillas):
    # Complete
    qc.barrier(state, *ancillas)
    return qc

qc_encode_bit = build_qc()

encode_bit_flip(qc_encode_bit, state_data, ancillas_data)

qc_encode_bit.draw(output="mpl")

### Prepare a decoding circuit
To readout our final state we must map it back from the codespace to a single qubit.
For our code this is simply $U_{de} = U_{en}^\dagger$.

This will be used to map our state out of the codespace
$$|\tilde{\Psi_1}\rangle \rightarrow |\Psi_1\rangle$$

In [ ]:
# Solution

def encode_bit_flip(qc, state, ancillas):
    control = state
    for ancilla in ancillas:
        qc.cx(control, ancilla)
    qc.barrier(state, *ancillas)
    return qc

qc_encode_bit = build_qc()

encode_bit_flip(qc_encode_bit, state_data, ancillas_data)

qc_encode_bit.draw(output="mpl")

In [ ]:
# Exercise

# Complete the routine below to decode from the bit-flip codespace
def decode_bit_flip(qc, state, ancillas):
    #complete
    
    return qc

qc_decode_bit = build_qc()

qc_decode_bit = decode_bit_flip(qc_decode_bit, state_data, ancillas_data)

qc_decode_bit.draw(output="mpl")

### A circuit that prepares our encoded state $|\tilde{\Psi}_1\rangle$

Below we see how we can combine the state preparation and encoding steps
to encode our state in the bit-flip code.

In [ ]:
qc_encoded_state_bit = qc_init.compose(qc_encode_bit)
qc_encoded_state_bit.draw(output="mpl")

In [ ]:
# Solution

def decode_bit_flip(qc, state, ancillas):
    inv = qc_encode_bit.inverse()
    return qc.compose(inv)

qc_decode_bit = build_qc()

qc_decode_bit = decode_bit_flip(qc_decode_bit, state_data, ancillas_data)

qc_decode_bit.draw(output="mpl")

### Measuring the syndome



However if we consider the action of bit-flip error $X$ which maps $|0\rangle \rightarrow |1\rangle$ and $|1\rangle -> |0\rangle$ on any of our qubits we have $\epsilon = \{E_0, E_1, E_2 \} = \{IIX, IXI, XII\}$ 


The circuit below measures the stabilizer $IZZ$ onto `creg_measure[0]` and $ZIZ$ onto `creg_measure[1]`. This effectively measures the parity of the two respective qubits in each stabilizer. 

- If we observe `IZZ= -1` an error ocurred on qubit 0 or 1
- If we observe `ZIZ= -1` an error ocurred on qubit 0 or 2

One important detail to note in the circuit below is that we *reset* our ancilla qubits after measuring the stabilizer. This is done so that we may reuse them for repeated stabilizer measurements. Also note we are making
use of the fact that we have already observed the state of the qubit and writing the conditional reset protocol
directly to avoid another round of qubit measurement if we used the `reset` instruction.

The circuit below will measure our Stabilizers generating a syndrome. We assume it is applied after some error channel $\Omega$ which takes $\Omega(|\tilde{\Psi_1}\rangle \langle\tilde{\Psi_1}|) = |\tilde{\Psi_2}\rangle \langle\tilde{\Psi_2}|$.
$$\langle \tilde{\Psi}_2|g_0\cdots g_n|\tilde{\Psi}_2\rangle \rightarrow |\tilde{\Psi}_2\rangle,\ \  \vec{x}$$

In [ ]:
# Exercise

# Complete the syndome measurement routine below
def measure_syndrome_bit(qc, qreg_data, qreg_measure, creg_measure):
    # Complete
    qc.barrier(*qreg_data, *qreg_measure)
    return qc

qc_syndrome_bit = measure_syndrome_bit(build_qc(), qreg_data, qreg_measure, creg_syndrome)
qc_syndrome_bit.draw(output="mpl")    

### Visualize our syndrome measurement

In [ ]:
qc_measure_syndrome_bit = qc_encoded_state_bit.compose(qc_syndrome_bit)
qc_measure_syndrome_bit.draw(output="mpl")

## Decoding and applying our correction sequence

Collectively measuring the stabilizers this is enough information to identify where a single X-flip error ocurred.
- If we measure `IZZ=-1` and `ZIZ=1` the error ocurred on qubit 1
- If we measure `IZZ=1` and `ZIZ=-1` the error ocurred on qubit 2
- If we measure `IZZ=-1` and `ZIZ=-1` the error ocurred on qubit 0

The circuit below corrects our state in the case of a single bit-flip error $$|\tilde{\Psi}_2\rangle \rightarrow |\tilde{\Psi}_1\rangle$$ 

In [ ]:
# Solution

def measure_syndrome_bit(qc, qreg_data, qreg_measure, creg_measure):
    def branch_delay():
        if sim:
            qc.barrier(qreg_data)
            qc.delay(block_branch_cycles, qreg_data)
            qc.barrier(qreg_data)
  
    qc.cx(qreg_data[0], qreg_measure[0])
    qc.cx(qreg_data[1], qreg_measure[0])
    qc.cx(qreg_data[0], qreg_measure[1])
    qc.cx(qreg_data[2], qreg_measure[1])
    qc.measure(qreg_measure, creg_measure)
    branch_delay() # grouped in hardware
    qc.x(qreg_measure[0]).c_if(creg_measure[0], 1)
    qc.x(qreg_measure[1]).c_if(creg_measure[1], 1)
    qc.barrier(*qreg_data, *qreg_measure)
    return qc

qc_syndrome_bit = measure_syndrome_bit(build_qc(), qreg_data, qreg_measure, creg_syndrome)
qc_syndrome_bit.draw(output="mpl")    

In [ ]:
# Exercise

# Complete the routine below to apply our decoding and correction sequence
def apply_correction_bit(qc, qreg_data, creg_syndrome):
    # Complete
    qc.barrier(qreg_data)
    return qc
    
qc_correction_bit = apply_correction_bit(build_qc(), qreg_data, creg_syndrome)
qc_correction_bit.draw(output="mpl")

In [ ]:
def apply_final_readout(qc, qreg_data, creg_data):
    """Apply inverse mapping so that we always try and measure |1> in the computational basis.
    
    TODO: The above is just a stand in for proper measurement basis measurement
    """
    qc.barrier(qreg_data)
    qc.measure(qreg_data, creg_data)
    return qc

qc_final_measure = apply_final_readout(build_qc(), qreg_data, creg_data)
qc_final_measure.draw(output="mpl")

## Below we see a complete cycle of the bit-flip code
We complete putting our building blocks together in the circuit below

In [ ]:
bit_code_circuit = qc_measure_syndrome_bit.compose(qc_correction_bit).compose(qc_final_measure)
bit_code_circuit.draw(output="mpl")

## Let's create a routine to perform multiple cycles of parity checks
The routine below will allow us to easily compose multiple cycles of our code interspersed
by the error channels provided in `qc_channels`.

In [ ]:
# Solution

def apply_correction_bit(qc, qreg_data, creg_syndrome):
    # If simulating we need to insert a delay to mirror the hardware
    def branch_delay():
        if sim:
            qc.barrier(qreg_data)
            qc.delay(block_branch_cycles, qreg_data)
            qc.barrier(qreg_data)
    
    branch_delay()
    qc.x(qreg_data[0]).c_if(creg_syndrome, 3)
    branch_delay()
    qc.x(qreg_data[1]).c_if(creg_syndrome, 1)
    branch_delay()
    qc.x(qreg_data[2]).c_if(creg_syndrome, 2)
    qc.barrier(qreg_data)
    return qc
    
qc_correction_bit = apply_correction_bit(build_qc(), qreg_data, creg_syndrome)
qc_correction_bit.draw(output="mpl")

In [ ]:
def build_error_correction_sequence(
    qc_base: QuantumCircuit,
    qc_init: Optional[QuantumCircuit], 
    qc_encode: QuantumCircuit, 
    qc_channels: List[QuantumCircuit],
    qc_syndrome: QuantumCircuit,
    qc_correct: QuantumCircuit,
    qc_decode: Optional[QuantumCircuit] = None,
    qc_final: Optional[QuantumCircuit] = None,
    name=None,
) -> QuantumCircuit:
    """Build a typical error correction circuit"""
    qc = qc_base
    
    if qc_init:
        qc = qc.compose(
            qc_init
        )
    
    qc = qc.compose(
            qc_encode
        )
    if name is not None:
        qc.name = name
    
    if not qc_channels:
        qc_channels = [QuantumCircuit(*qc.qregs)]
        
    for qc_channel in qc_channels:
        qc = qc.compose(
                qc_channel
            ).compose(
                qc_syndrome
            ).compose(
                qc_correct
            )
    if qc_decode:
        qc = qc.compose(qc_decode)
    
    if qc_final:
        qc = qc.compose(qc_final)
    
    return qc

For example

In [ ]:
bit_code_circuit = build_error_correction_sequence(
    build_qc(),
    qc_init,
    qc_encode_bit,
    [],
    qc_syndrome_bit,
    qc_correction_bit,
    None,
    qc_final_measure,
)
bit_code_circuit.draw(output="mpl")

## Running on Hardware

We now will choose a qubit layout on the device and transpile our circuit to a realizeable
circuit on the device. We will see that due to limited connectivity we will have to perform
qubit routing. As not all qubits are created equal it is important to choose good sets of qubits
in the hardware.

In [ ]:
layout_circuit = transpile(bit_code_circuit, backend, optimization_level=3)
print(initial_layout := calculate_initial_layout(layout_circuit, False, favourite_qubits))

In [ ]:
transpiled_bit_code_circuit = apply_transpile(bit_code_circuit, backend, initial_layout=initial_layout)
transpiled_bit_code_circuit.draw(output="mpl")

## Execute the circuit in hardware
Below we execute the circuit in the hadware and then decode the execution results.

In [ ]:
result = execute(transpiled_bit_code_circuit).result()

In [ ]:
def decode_result(data_counts, syndrome_counts, verbose=True, indent=0):
    shots = sum(data_counts.values())
    success_trials = data_counts.get('000', 0) + data_counts.get('111', 0)
    failed_trials = shots-success_trials
    error_correction_events = shots-syndrome_counts.get('00', 0)
    
    if verbose:
        print(f"{' ' * indent}Bit flip errors were corrected on {error_correction_events}/{shots} trials")
        print(f"{' ' * indent}A final parity error was detected on {failed_trials}/{shots} trials")
        print(f"{' ' * indent}No error was detected on {success_trials}/{shots} trials")
    return error_correction_events, failed_trials

data_indices = list(range(len(qreg_data)))
syndrome_indices = list(range(data_indices[-1]+1, len(qreg_data) + len(qreg_measure) ))
                    
marginalized_data_result = marginal_counts(result, data_indices)
marginalized_syndrome_result = marginal_counts(result, syndrome_indices)

print(f'Completed bit code experiment data measurement counts {marginalized_data_result.get_counts(0)}')
print(f'Completed bit code experiment syndrome measurement counts {marginalized_syndrome_result.get_counts(0)}')
decode_result(marginalized_data_result.get_counts(0), marginalized_syndrome_result.get_counts(0));

## Let's emulate a random error source
Here we will use some more control flow to insert a random bit-flip error by using an ancilla qubit as a source of random bit flips and then look at the performance of our code.

In [ ]:
from qiskit.circuit.library import IGate, XGate, ZGate

qreg_error_ancilla = QuantumRegister(1)
creg_error_ancilla = ClassicalRegister(1)

def build_random_error_channel(gate, ancilla, creg_ancilla, error_qubit):
    """Build an error channel that randomly applies a single-qubit gate based on an ancilla qubit measurement result"""
    qc = build_qc()
    qc.add_register(qreg_error_ancilla)
    qc.add_register(creg_error_ancilla)
    qc.barrier(ancilla, error_qubit.register)
    # 50-50 chance of applying a bit-flip
    qc.h(ancilla)
    qc.measure(ancilla, creg_ancilla)
    qc.append(gate, [error_qubit]).c_if(creg_ancilla, 1)
    qc.barrier(ancilla, error_qubit.register)
    return qc

qc_id_error_channel = build_random_error_channel(IGate(), qreg_error_ancilla, creg_error_ancilla, qreg_data[0])
print("Identity error channel")
print(qc_id_error_channel.draw(idle_wires=False,output='text',fold=-1))

qc_bit_flip_error_channel = build_random_error_channel(XGate(), qreg_error_ancilla, creg_error_ancilla, qreg_data[0])
print("Bit flip error channel")
print(qc_bit_flip_error_channel.draw(idle_wires=False,output='text',fold=-1))

qc_phase_flip_error_channel = build_random_error_channel(ZGate(), qreg_error_ancilla, creg_error_ancilla, qreg_data[0])
print("Phase flip error channel")
print(qc_phase_flip_error_channel.draw(idle_wires=False,output='text',fold=-1))

## Let us now add our error channel in between our syndrome measurement and decode/correct phase to emulate a source of error

In [ ]:
def build_error_channel_base():
    qc = build_qc()
    qc.add_register(qreg_error_ancilla)
    qc.add_register(creg_error_ancilla)
    return qc

qc_id_error_bit_flip_code = build_error_correction_sequence(
    build_error_channel_base(),
    qc_init,
    qc_encode_bit,
    [qc_id_error_channel],
    qc_syndrome_bit,
    qc_correction_bit,
    None,
    qc_final_measure,
    "Identity error channel"
)

qc_bit_flip_error_bit_flip_code = build_error_correction_sequence(
    build_error_channel_base(),
    qc_init,
    qc_encode_bit,
    [qc_bit_flip_error_channel],
    qc_syndrome_bit,
    qc_correction_bit,
    None,
    qc_final_measure,
    "Bit flip error channel"
)

qc_phase_flip_error_bit_flip_code = build_error_correction_sequence(
    build_error_channel_base(),
    qc_init,
    qc_encode_bit,
    [qc_phase_flip_error_channel],
    qc_syndrome_bit,
    qc_correction_bit,
    None,
    qc_final_measure,
    "Phase flip error channel"
)

circuits_error_channels_bit_flip_code = [qc_id_error_bit_flip_code, qc_bit_flip_error_bit_flip_code, qc_phase_flip_error_bit_flip_code]

qc_bit_flip_error_bit_flip_code.draw(output="mpl")

## Execute the circuits

In [ ]:
# We need to add an extra ancilla qubit to our layout
# It doesn't matter which qubit for the must part as we are using as a
# source of random information
error_channel_layout = error_channel_layout = initial_layout + list(set(range(circuits_error_channels_bit_flip_code[0].num_qubits)) - set(initial_layout))[:1]

transpiled_circuits_error_channels_bit_flip_code = apply_transpile(circuits_error_channels_bit_flip_code, backend, initial_layout=error_channel_layout)
result_error_channels_bit_flip_code = execute(transpiled_circuits_error_channels_bit_flip_code).result()

In [ ]:
from qiskit.quantum_info.analysis import hellinger_fidelity

def decode_error_channel_result(qc_init, data_counts, syndrome_counts, verbose=True, indent=0):
    shots = sum(data_counts.values())
    logical_zero = data_counts.get('000', 0)
    logical_one = data_counts.get('111', 0)
    success_trials = logical_zero + logical_one
    failed_trials = shots-success_trials
    logical_counts = {"0": logical_zero, "1": logical_one}
    
    
    ideal_transpiled = apply_transpile(qc_init_outcome, backend, initial_layout)
    counts_ideal = marginal_counts(ideal_sim.run(ideal_transpiled, shots=success_trials).result().get_counts(0), indices=[0])
    
    fidelity = hellinger_fidelity(counts_ideal, logical_counts)
    
    error_correction_events = shots-syndrome_counts.get('00', 0)
    
    if verbose:
        print(f"{' ' * indent}Bit flip errors were corrected on {error_correction_events}/{shots} trials")
        print(f"{' ' * indent}A final parity error was detected on {failed_trials}/{shots} trials")
        print(f"{' ' * indent}For the successful trials the Hellinger fidelity is {fidelity}")
    return error_correction_events, failed_trials

## We see that we correct the majority of the bit flip errors introduced
Question: Why does the phase-flip error not create errors in the final outcome?

In [ ]:
qc_init_outcome = qc_init.copy()
qc_init_outcome.measure(qreg_data[0], 0)

qreg_indices = list(range(len(qreg_data)))
data_indices = qreg_indices[:1]
syndrome_indices = list(range(qreg_indices[-1]+1, len(qreg_data) + len(qreg_measure) ))

result_decoded_data_qubit_marginal_err_ch = marginal_counts(result_error_channels_bit_flip_code, data_indices)
result_data_marginal_err_ch = marginal_counts(result_error_channels_bit_flip_code, qreg_indices)
result_syndrome_marginal_err_ch = marginal_counts(result_error_channels_bit_flip_code, syndrome_indices)

for i, qc in enumerate(transpiled_circuits_error_channels_bit_flip_code):
    print(f"For {qc.name} with bit flip code")
    print(f'    Completed bit code experiment decoded data qubit measurement counts {result_decoded_data_qubit_marginal_err_ch.get_counts(i)}')
    print(f'    Completed bit code experiment data qubits measurement counts {result_data_marginal_err_ch.get_counts(i)}')
    print(f'    Completed bit code experiment syndrome measurement counts {result_data_marginal_err_ch.get_counts(i)}')
    decode_error_channel_result(qc_init_outcome, result_data_marginal_err_ch.get_counts(i), result_syndrome_marginal_err_ch.get_counts(i), indent=4);
    print("")

# Exercises
Now we will complete several exercises that build on the code above.

We will aim to evaluate the bit-flip code.

## Exercise #1 - Compare our bit-flip code with a code that applies the "identity" correction

Complete the method below ``apply_no_correction_bit`` to create a decoding sequence
that *does not* correct the error but rather performs conditional identity operations

In [ ]:
# Exercise

# Complete the routine below to apply an "identity" correction rather than
# the standard bit-flip correction sequence
def apply_no_correction_bit(qc, qreg_data, creg_syndrome):
    """Complete the method below to apply an "identity correction"
    
    Make sure to still apply the conditional gates so that the comparison to the bit-flip
    code is faithful.
    """
    # Complete
    qc.barrier(qreg_data)
    return qc

    
qc_no_correction_bit = apply_no_correction_bit(build_qc(), qreg_data, creg_syndrome)
qc_no_correction_bit.draw(output="mpl")

Complete the methods below similar to `qc_id_error_bit_flip_code` above
to implement circuits that *do not* correct the bit flip errors, using your routine above.

In [ ]:
# Exercise

# Complete the sequence using `build_error_correction_sequence` to evaluate the
# performance of the identity correction sequence

# Complete the following routines
qc_id_error_no_correct = QuantumCircuit() # build_error_correction_sequence(...)
qc_bit_flip_error_no_correct = QuantumCircuit() # build_error_correction_sequence(...)
qc_phase_flip_error_no_correct = QuantumCircuit() # build_error_correction_sequence(...)


circuits_error_channels_no_correct = [qc_id_error_no_correct, qc_bit_flip_error_no_correct, qc_phase_flip_error_no_correct]

qc_id_error_no_correct.draw(output="mpl")

Execute the circuits you created that *do not* perform a correction

In [ ]:
# We need to add an extra ancilla qubit to our layout
# It doesn't matter which qubit for the must part as we are using as a
# source of random information
error_channel_layout = error_channel_layout = initial_layout + list(set(range(circuits_error_channels_bit_flip_code[0].num_qubits)) - set(initial_layout))[:1]

transpiled_circuits_error_channels_no_correct = apply_transpile(circuits_error_channels_no_correct, backend, initial_layout=error_channel_layout)
result_error_channels_no_correct = execute(transpiled_circuits_error_channels_no_correct).result()

Analyze how well these circuits performed. Is the result worse than that of the corrected bit-flip code?

In [ ]:
qc_init_outcome = qc_init.copy()
qc_init_outcome.measure(qreg_data[0], 0)

qreg_indices = list(range(len(qreg_data)))
data_indices = qreg_indices[:1]
syndrome_indices = list(range(qreg_indices[-1]+1, len(qreg_data) + len(qreg_measure) ))

result_decoded_data_qubit_marginal_no_correct = marginal_counts(result_error_channels_no_correct, data_indices)
result_data_marginal_no_correct = marginal_counts(result_error_channels_no_correct, qreg_indices)
result_syndrome_marginal_no_correct = marginal_counts(result_error_channels_no_correct, syndrome_indices)

for i, qc in enumerate(transpiled_circuits_error_channels_no_correct):
    print(f"For {qc.name} with bit flip code")
    print(f'    Completed bit code experiment decoded data qubit measurement counts {result_decoded_data_qubit_marginal_no_correct.get_counts(i)}')
    print(f'    Completed bit code experiment data qubits measurement counts {result_data_marginal_no_correct.get_counts(i)}')
    print(f'    Completed bit code experiment syndrome measurement counts {result_data_marginal_no_correct.get_counts(i)}')
    decode_error_channel_result(qc_init_outcome, result_data_marginal_no_correct.get_counts(i), result_syndrome_marginal_no_correct.get_counts(i), indent=4);
    print("")

## Exercise #2 - Evaluate the performance of multiple stabilizer cycles 

We will now evaluate multiple cycles of measuring the stabilizer and applying our correction sequence using the routines we have created above. For reference we will create a circuit which prepares our state and *does not*
encode it to compare our performance with a raw physical qubit.

In this exercise we will plot the performance (Hellinger fidelity) of our bit flip code as a function of the number of stabilizer cycles. 

In [ ]:
def apply_final_readout_invert(qc, qc_init, qreg_data, creg_data):
    """Apply inverse mapping so that we always try and measure |1> in the computational basis."""
    qc = qc.compose(qc_init.inverse())
    qc.x(qreg_data[0])
    qc.barrier(qreg_data)
    qc.measure(qreg_data, creg_data)
    return qc

qc_final_measure_invert = apply_final_readout_invert(build_qc(), qc_init, qreg_data, creg_data)
qc_final_measure_invert.draw(output="mpl")

In [ ]:
# Solution

qc_id_error_no_correct = build_error_correction_sequence(
    build_error_channel_base(),
    qc_init,
    qc_encode_bit,
    [qc_id_error_channel],
    qc_syndrome_bit,
    qc_no_correction_bit,
    None,
    qc_final_measure,
    "Identity error channel"
)

qc_bit_flip_error_no_correct = build_error_correction_sequence(
    build_error_channel_base(),
    qc_init,
    qc_encode_bit,
    [qc_bit_flip_error_channel],
    qc_syndrome_bit,
    qc_no_correction_bit,
    None,
    qc_final_measure,
    "Bit flip error channel"
)

qc_phase_flip_error_no_correct = build_error_correction_sequence(
    build_error_channel_base(),
    qc_init,
    qc_encode_bit,
    [qc_phase_flip_error_channel],
    qc_syndrome_bit,
    qc_no_correction_bit,
    None,
    qc_final_measure,
    "Phase flip error channel"
)

circuits_error_channels_no_correct = [qc_id_error_no_correct, qc_bit_flip_error_no_correct, qc_phase_flip_error_no_correct]

qc_id_error_no_correct.draw(output="mpl")

For comparison with a "raw" physical qubit we include the routine below which builds a circuit that idles instead of encoding and correcting our qubits. You should include this in your sweep.

In [ ]:
from collections import defaultdict

from qiskit.transpiler import PassManager, InstructionDurations
from qiskit.transpiler.passes import ASAPSchedule, PadDelay
from qiskit_ibm_provider.transpiler.passes.scheduling import DynamicCircuitScheduleAnalysis, PadDelay

def get_circuit_duration_(qc: QuantumCircuit) -> int:
    """Get duration of circuit in hardware cycles."""
    durations = InstructionDurations.from_backend(backend)
    pm = PassManager([DynamicCircuitScheduleAnalysis(durations)])
    pm.run(qc)
    node_start_times = pm.property_set["node_start_time"]
    
    block_durations = defaultdict(int)
    for inst, (block, t0) in node_start_times.items():
        block_durations[block] = max(block_durations[block], t0+inst.op.duration)
        
    duration = sum(block_durations.values())
    
    # If we are running on real hardware the delays have not been appended to the
    # circuit directly and are instead built into the conditional operations.
    # We account for them manually here
    if not sim:
        duration += block_branch_cycles * (len(block_durations) - 1)
    return duration

def build_idle_error_correction_sequence(
    qc_base: QuantumCircuit,
    qc_init: Optional[QuantumCircuit], 
    qc_encode: QuantumCircuit, 
    qc_channels: List[QuantumCircuit],
    qc_syndrome: QuantumCircuit,
    qc_correct: QuantumCircuit,
    qc_decode: Optional[QuantumCircuit] = None,
    qc_final: Optional[QuantumCircuit] = None,
    initial_layout=initial_layout,
    name: str = None,
) -> QuantumCircuit:
    """Build a quantum circuit that idles for the period of the input error correction sequence."""
    qc = qc_base
    
    if qc_init:
        qc = qc.compose(
            qc_init
        )
    if name is not None:
        qc.name = name

    qc_idle_region = qc_base.copy()
    
    qc_idle_region.compose(
            qc_encode
        )
  
    if not qc_channels:
        qc_channels = [QuantumCircuit(*qc.qregs)]
        
    for qc_channel in qc_channels:
        qc_idle_region = qc_idle_region.compose(
                qc_channel
            ).compose(
                qc_syndrome
            ).compose(
                qc_correct
            )
    
    if qc_decode:
        qc_idle_region = qc_idle_region.compose(qc_decode)
    
    qc_idle_transpiled = apply_transpile(qc_idle_region, backend, initial_layout=initial_layout, scheduling_method=None)
    idle_duration = get_circuit_duration_(qc_idle_transpiled)
    
    qc_idle = qc_base.copy()
    qc_idle.barrier()
    for qubit in qc_idle.qubits:
        qc_idle.delay(idle_duration, qubit, unit="dt")
    qc_idle.barrier()
    qc = qc.compose(qc_idle)
    
    if qc_final:
        qc = qc.compose(qc_final_measure)
    
    return qc

## Exercise #3 - Evaluate the performance of an "identity" correction sequence

For reference we will now evaluate the performance of an identity correction sequence.

That is to say where we apply our correction only with conditional `id` gates.

### 3a. Complete the routine below to create an "idle" error channel

In [ ]:
# Exercise

def build_idle_error_channel(time_in_seconds, qreg):
    idle_cycles = convert_cycles(time_in_seconds, backend)
    qc_idle = build_qc()
    # Complete
    qc_idle.barrier()
    return qc_idle

In [ ]:
# Solution

def build_idle_error_channel(time_in_seconds, qreg):
    idle_cycles = convert_cycles(time_in_seconds, backend)
    qc_idle = build_qc()
    qc_idle.delay(idle_cycles, qreg, "dt")
    qc_idle.barrier()
    return qc_idle

### 3b. Complete the routine below to perform a sweep of our bit-flip, no-correction, and idle circuits for a number of iterations.

*hint*: Make sure to use the routines `build_error_correction_sequence` and `build_idle_error_correction_sequence`.

In [ ]:
# Exercise

num_rounds = 5

# Idle for a specified period in seconds
# This is how we build an idle error channel
idle_period = 5e-6

# Use the circuit we created above
qc_idle = build_idle_error_channel(idle_period, qreg_data)


qcs_corr_bit = []
qcs_no_corr_bit = []
qcs_idle_equiv_bit = []
for round_ in range(num_rounds):
    qc_error_channels = [qc_idle] * (round_ + 1)
    
    # Complete bit flip code iteration
    qcs_corr_bit.append(QuantumCircuit())
    # Complete no correction bit flip code iteration
    qcs_no_corr_bit.append(QuantumCircuit())
    # Complete idle equivalent bit flip code iteration
    qcs_idle_equiv_bit.append(QuantumCircuit())



Below we execute and plot the circuit we constructed.

In [ ]:
qcs_corr_bit[0].draw(output="mpl", idle_wires=False)

In [ ]:
transpiled_qcs_corr_bit = apply_transpile(qcs_corr_bit, backend, initial_layout=initial_layout)
job_qcs_corr_bit = execute(transpiled_qcs_corr_bit)

transpiled_qcs_no_corr_bit = apply_transpile(qcs_no_corr_bit, backend, initial_layout=initial_layout)
job_qcs_no_corr_bit = execute(transpiled_qcs_no_corr_bit)

transpiled_qcs_idle_equiv_bit = apply_transpile(qcs_idle_equiv_bit, backend, initial_layout=initial_layout, dynamical_decoupling=True)
job_qcs_idle_equiv_bit = execute(transpiled_qcs_idle_equiv_bit)

result_qcs_corr_bit = job_qcs_corr_bit.result()
result_qcs_no_corr_bit = job_qcs_no_corr_bit.result()
result_qcs_idle_equiv_bit = job_qcs_idle_equiv_bit.result()

In [ ]:
transpiled_qcs_idle_equiv_bit[-1].draw(output="mpl")

In [ ]:
from qiskit.quantum_info.analysis import hellinger_fidelity

qc_init_outcome = qc_init.copy()
qc_init_outcome = qc_init_outcome.compose(qc_final_measure)
result_ideal = ideal_sim.run(apply_transpile(qc_init_outcome, backend, initial_layout)).result()

def calculate_hellinger_fidelity(result_ideal, result_experiment, data_qubit=0):
    result_ideal = marginal_counts(result_ideal, indices=[data_qubit])
    result_experiment = marginal_counts(result_experiment, indices=[data_qubit])
    
    counts_ideal = result_ideal.get_counts(0)

    hellinger_fidelities = []
    for circuit_idx in range(len(result_experiment.results)):
        hellinger_fidelities.append(hellinger_fidelity(counts_ideal, result_experiment.get_counts(circuit_idx)))

    return hellinger_fidelities

### 3c. Use the routuine `calculate_hellinger_fidelity` above to extract the Hellinger fidelity for our sweeps

In [ ]:
# Evaluate the Hellinger fidelity using the routine above
fidelities_corr_bit = 0.
fidelities_no_corr_bit = 0.
fidelities_idle_equiv_bit = 0.

In [ ]:
# Solution

num_rounds = 5

# Idle for a specified period in seconds
# This is how we build an idle error channel
idle_period = 5e-6

# Use the circuit we created above
qc_idle = build_idle_error_channel(idle_period, qreg_data)


qcs_corr_bit = []
qcs_no_corr_bit = []
qcs_idle_equiv_bit = []
for round_ in range(num_rounds):
    qc_error_channels = [qc_idle] * (round_ + 1)
    
    #qcs_corr_bit.append(QuantumCircuit())

    qcs_corr_bit.append(
        build_error_correction_sequence(
            build_qc(),
            qc_init,
            qc_encode_bit,
            qc_error_channels,
            qc_syndrome_bit,
            qc_correction_bit,
            qc_decode_bit,
            qc_final_measure_invert,
            f"With Correction {round_}"
        )
    )
    #qcs_no_corr_bit.append(QuantumCircuit())
    qcs_no_corr_bit.append(
        build_error_correction_sequence(
            build_qc(),
            qc_init,
            qc_encode_bit,
            qc_error_channels,
            qc_syndrome_bit,
            qc_no_correction_bit,
            qc_decode_bit,
            qc_final_measure_invert,
            name=f"Without Correction {round_}"
        )
    )
    
    #qcs_idle_equiv_bit.append(QuantumCircuit())
    qcs_idle_equiv_bit.append(
        build_idle_error_correction_sequence(
            build_qc(),
            qc_init,
            qc_encode_bit,
            qc_error_channels,
            qc_syndrome_bit,
            qc_correction_bit,
            qc_decode_bit,
            qc_final_measure_invert,
            initial_layout=initial_layout,
            name=f"Idle {round_}"
        )
    )


In [ ]:
fidelities_corr_bit = calculate_hellinger_fidelity(result_ideal, result_qcs_corr_bit)
fidelities_no_corr_bit = calculate_hellinger_fidelity(result_ideal, result_qcs_no_corr_bit)
fidelities_idle_equiv_bit = calculate_hellinger_fidelity(result_ideal, result_qcs_idle_equiv_bit)

Below we plot the performance of our error correcting code as a function of the number of iterations.

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12, 6)

iters = range(1, num_rounds+1)
plt.plot(iters, fidelities_corr_bit, label="Bit flip code - correction")
plt.plot(iters, fidelities_no_corr_bit, label="Bit flip code - no correction")
plt.plot(iters, fidelities_idle_equiv_bit, label="Idle equivalent circuit")
plt.ylabel("Hellinger Fidelity")
plt.xlabel("Correction cycles")
plt.xticks(iters)
plt.suptitle("Comparing the performance of error-correction strategies for multiple correction cycles")
plt.title(f"Idle period: {idle_period*1e6}us - Qubit layout: {initial_layout} - Dynamical decoupling: {dynamical_decoupling}")
plt.legend(loc="upper right")

## Exercise #4 - Evaluate the performance of protecting different input states

Thus far we have only been preparing the state $|001\rangle$ and encoding it as $|111\rangle$.
What we *actually* care about is the average quality of protecting *any* input state.
Rerun your analysis code above but instead of initializing with the `|1>` state,
select a different state.

**Note**: Make sure to save your results before overwriting them. Either save your plots or create a copy of this notebook.

- How do other states perform? 
- Does the quality improve or get worse?
- Why might we expect the `|1>` to perform the best?
- What sorts of noise might be affecting the qubits?

In [181]:
result_qcs_corr_bit = job_qcs_corr_bit.result()
result_qcs_no_corr_bit = job_qcs_no_corr_bit.result()
result_qcs_idle_equiv_bit = job_qcs_idle_equiv_bit.result()

## Exercise #5 [Stretch Goal] [[4,1,2]] circuits

In [ ]:
def build_412(circuit,qubits,parity,layout='zxz') -> QuantumCircuit:
    ### qubits
    # qubits: list of 7 qubits 
    # ex. [q[0],q[1],q[2],q[3],q[4],q[5],q[6],q[7]] 
    #where first 4 are data qubits, 5 and 6 are flags, and 7 is ancilla (see image from heavy-hex paper)
    ### parity
    # parity is 'x' or 'z'
    ### initialized_ancilla = True or False. 
    # If True, will prepare f1,f2,a1 in correct basis and apply post rotation before measurement 
    # Will include barrier after prep and before post rot if True
    [d1,d2,d3,d4,f1,f2,a1]=qubits
    
    if layout == 'zxz':
        if parity == 'x':
            circuit.h(a1)
            circuit.cx(a1,f2)
            circuit.cx(a1,f1)
            circuit.cx(f2,d1)
            circuit.cx(f1,d4)
            circuit.cx(f2,d2)
            circuit.cx(f1,d3)
            circuit.cx(a1,f2)
            circuit.cx(a1,f1)
            circuit.h(a1)           
        if parity == 'z':
            circuit.cx(d2,f2)
            circuit.cx(d1,f2)
            circuit.cx(d3,f1)
            circuit.cx(d4,f1)
        return circuit
    
    elif layout == 'xzx':
        if parity == 'z':
            circuit.h(f1)
            circuit.h(f2)

            circuit.cx(f2,a1)
            circuit.cx(f1,a1)
            circuit.cx(d1,f2)        
            circuit.cx(d3,f1)
            circuit.cx(d2,f2)
            circuit.cx(d4,f1)
            circuit.cx(f2,a1)
            circuit.cx(f1,a1)

            circuit.h(f1)
            circuit.h(f2)

        if parity == 'x':
            circuit.h(f1)
            circuit.h(f2)
            circuit.cx(f2,d2)
            circuit.cx(f2,d1)
            circuit.cx(f1,d3)
            circuit.cx(f1,d4)

            circuit.h(f1)
            circuit.h(f2)
        return circuit        

In [ ]:
# Complete - how do you do the Z and X checks for this code? How do you decode the errors?
# See: https://arxiv.org/abs/2110.04285


In [ ]:
import qiskit.tools.jupyter
%qiskit_version_table
%qiskit_copyright

## Exercise #5 - Rerun with dynamical decoupling enabled

Dynamical decoupling is a technique which can decouple a qubit from neighboring
quantum systems during its evolution [0](https://arxiv.org/abs/quant-ph/9803057). 
Qiskit supports adding dynamical decoupling by identifying idle sequences in which
identity sequences (such as $\tau/4-X-\tau/2-X-\tau/4$) may be automatically inserted
by a [specialized pass](https://github.com/Qiskit/qiskit-ibm-provider/blob/06490ab5ccfabc1ceae79bc3e8ba3f8bb82296fa/qiskit_ibm_provider/transpiler/passes/scheduling/dynamical_decoupling.py#L32).

This notebook already has dynamical decoupling support built-in to `apply_transpile`.
To enable it set `dynamical_decoupling=True` and rerun the notebook.


**Note**: Make sure to save your results before overwriting them. Either save your plots or create a copy of this notebook.

- How does this perform for your state $|1>$? What about other states?
- What constraints does dynamical decoupling have that might make it more difficult to implement in dynamic circuits? 